# A Complete Solution to the BackBaze.com Kaggle Problem

## Step Five.  
### Modeling the data.

## Table of contents

1. [Introduction](#10)<br>

2. [Establish environment and parameters](#20)<br>
3. [Create a target window.](#30)<br>
4. [Create training, testing and validation groups ](#40)<br>
5. [Prepare the data for the model ](#50)<br>
6. [Build the model ](#60)<br>
7. [Evaluate model](#70)<br>
8. [Format the scored data for further evaluation](#80)<br>
9. [Export the data for use in Step 6](#90)<br>

### 1.0 Introduction <a id="10"></a>

Note this is part five of a six-part solution.

BackBlaze.com, you are the "GOAT." You are the "cat's meow." You "Rock the House." In case you don't know why BackBaze.com is so totally "kick-ass," they open-sourced a vast set of hard drive information a few years ago and continue updating it each quarter.  What a treasure trove of superb data.  BackBlaze.com, thank you from the bottom of my heart.

The backblaze.com data includes operational metrics from hard drives with an indicator of a hard-drive failure.  It is an excellent source for teaching techniques related to machine failure.  Again, thank you for making this available to the open-source community.
Here is a link to the data.

https://www.backblaze.com/b2/hard-drive-test-data.html

My goal in this series of articles is not to give the best solution with the highest AUC.  My goal is to show you how to approach equipment failure problems and build solutions that reflect realistic accuracy, and provide an easy transition from the lab to the real world.

I will use a Spark/Python Jupyter notebook inside IBM's Watson Studio on the cloud as a tool in this discussion.

https://www.ibm.com/cloud/watson-studio

I will also be using cloud object storage on the IBM cloud.

https://www.ibm.com/cloud/block-storage





To be fully transparent, I know very little about the operation and management of disk drives.  I am not a subject matter expert in disk drives.  My lack of expertise is critical to note because developing data science solutions usually requires some knowledge of the subject matter.  Because of this lack of knowledge, I am sure this solution is lacking in some areas.  Despite this, I have made assumptions when necessary, documented gaps in my understanding, and plowed through the problem from beginning to end.   Consider this work a template to solve the problem, not a solution specifically built for Backblaze to operationalize immediately.  Undoubtedly, I would need to consult with disk drive experts before finalizing the solution.

In the fifth article of the series, I create build a model.

### 2.0 Establish environment and Parameters <a id="20"></a>

Establish Libraries

In [2]:
from functools import reduce
from pyspark.sql import DataFrame

import pyspark.sql.functions as F
from pyspark.sql.functions import *

import pandas as pd

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix
from pyspark.ml.classification import GBTClassifier
import numpy as np

spark.conf.set("spark.sql.broadcastTimeout",  7200000)
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")

Define Cloud Object Storage Credentials

In [3]:
# The code was removed by Watson Studio for sharing.

Read in Data

In [4]:
df = spark.read.parquet(cos.url('data_2020_model.parquet', 'backblazedata-donotdelete-pr-cij57grgkoctem'))
print((df.count(), len(df.columns)))

(545127, 233)


### 3.0 Create a target window. <a id="30"></a>

Typically, industrial machines fail infrequently.  Manufacturers design and build them to last and perform reliably over time.  Reliable machines mean we have few incidences of failure to study and model. One standard way of dealing with a small number of failures is to expand the failure window.  Instead of predicting the specific day of failure in our current use case, predict the days leading up to failure.  Expanding the failure window makes sense.  When a machine fails, it rarely just happens.   It is rare for a machine to be perfect on Monday, terrible on Tuesday, and then perfect on Wednesday without some intervention.  Machines, like humans, wear down over time.  Because of this,  we can incorporate the days before a failure as part of the dependent variable.

There is no magic in determining the length of the failure window.  The only hard rule is that it needs to fit the context of the problem.  For example, if a machine lasts 20 years, a 5-second failure window is a little short.  If it fails every twenty years, a window of three, six, or nine months is likely more reasonable.  Likewise, a six-day failure window does not make much sense if a machine fails twice a day.

In the current use case, I used an eleven-day failure window.  Again, there is nothing magical about that.  I chose it because it makes sense.

Again, there is a lot about disk drives that I don't know.  I am not a subject matter expert regarding disk drives and storage.  Someone with more expertise in the space may argue for a larger or smaller window.  Also, the target window is a parameter that you can tune.  We may get better results with a larger or smaller window.  I did not attempt to adjust the window size but may do some time in the future.

Create a data frame with only records of failure.

In [61]:
df_failure=df.filter(df.FAILURE ==1)

Limit the fields

In [6]:
df_failure=df_failure[['DATE','SERIAL_NUMBER','FAILURE']]

Drop the failure field and rename the date field as the failure date.

In [7]:
df_failure = df_failure.drop("FAILURE")
df_failure=df_failure.withColumnRenamed("DATE","FAILURE_DATE")
df_failure=df_failure.withColumnRenamed("SERIAL_NUMBER","SERIAL_NUMBERX")

In [8]:
#df_failure.show(5)

Ensure you only have one failure per serial number.

In [9]:
from pyspark.sql import Window
from pyspark.sql.functions import rank
window = Window.partitionBy("SERIAL_NUMBERX").orderBy("FAILURE_DATE")
df_failure=(df_failure.withColumn('rank', rank().over(window))
.filter(col('rank') == 1)
.drop('rank'))

Left join the list of failures to the original data frame.  The result is a complete list of disks with a failure.

Keep a list of disks with a failure. We will use this later on.

In [11]:
df_failure_list=df_failure[['SERIAL_NUMBERX']]
df_failure_list = df_failure_list.withColumn("wookie", lit(1))

In [12]:
df_failure=df.join(df_failure,(((df.SERIAL_NUMBER) ==  (df_failure.SERIAL_NUMBERX)) ),"inner")
df_failure = df_failure.drop("SERIAL_NUMBERX")

In [13]:
#print((df_failure.count(), len(df_failure.columns)))

Select relevant fields and calculate the difference between the current date and the failure date

In [14]:
from pyspark.sql.functions import *
dater=df_failure.select(
      col("SERIAL_NUMBER"),
      col("DATE"),
      col("FAILURE_DATE"),
      datediff(col("FAILURE_DATE"),col("DATE")).alias("datediff"))

create a field called "TARGET" that is 1 if the date is with 11 days of a failure

In [15]:
#dater.show(1000)

In [16]:
from pyspark.sql.functions import when

dater=dater.withColumn("TARGET", when(dater['datediff'] <=10, 1).otherwise(0))


#dater.show(1000)

Rename the data frame.

In [17]:
dater=dater.withColumnRenamed("SERIAL_NUMBER","SERIAL_NUMBERX")
dater=dater.withColumnRenamed("DATE","DATEX")
dater=dater.withColumnRenamed("FAILURE_DATE","FAILURE_DATEX")

In [18]:
#df_failure.show(1)

In [19]:
df_failure=df_failure.join(dater,(((df_failure.SERIAL_NUMBER) ==  (dater.SERIAL_NUMBERX)) & ((df_failure.DATE)==(dater.DATEX)) & ((df_failure.FAILURE_DATE)==(dater.FAILURE_DATEX))),"inner")
#drop dummy keys
df_failure = df_failure.drop("SERIAL_NUMBERX")
df_failure = df_failure.drop("DATEX")
df_failure = df_failure.drop("FAILURE_DATEX")

In [20]:
#print((df_failure.count(), len(df_failure.columns)))

Join the list of failure disks to the original data frame, keep only disks that do not have a failure.

In [21]:
#join the list of failure directories to the original data frame with a left join
df_nonfailure=df.join(df_failure_list,(((df.SERIAL_NUMBER) ==  (df_failure_list.SERIAL_NUMBERX)) ),"left")
#drop dummy key
df_nonfailure = df_nonfailure.drop("SERIAL_NUMBERX")
#assign dummy variable to indicate disk did not fail in 2020
df_nonfailure=df_nonfailure.na.fill(0,["wookie"]) 
#select only disks that failed
df_nonfailure=df_nonfailure.filter(df_nonfailure.wookie ==0)
#assign the predicted variable of 0
df_nonfailure = df_nonfailure.withColumn("TARGET", lit(0))
#clean things up
df_nonfailure = df_nonfailure.drop("wookie")



Create a failure date way in the future for the nonfailure data

In [22]:
df_nonfailure = df_nonfailure.withColumn("FAILURE_DATE", expr("make_date(2099, 12, 12)"))

In [23]:
#df_nonfailure.show()

In [24]:
#print((df_nonfailure.count(), len(df_nonfailure.columns)))

In [25]:

df_failure = df_failure.drop("datediff")

In [26]:
#df_failure.show(1)

In [27]:
df = df_failure.unionByName(df_nonfailure)

In [28]:
z=df_failure.groupBy("TARGET").count()
z.show()

+------+------+
|TARGET| count|
+------+------+
|     1| 11000|
|     0|181789|
+------+------+



In [29]:
#df.show(1)

### 4.0 Create training, testing and validation groups <a id="40"></a>

Next, we will need to create a Training, Testing and Validation group for our modeling process.  In this use case and in most equipment failure problems, we can not randomly assign records to each group.  Because the data is a cross-sectional time-series, we must define the Training, Testing and Validation groups based on serial number.  I wrote about the reasons for this extensively in another article. For more information, please the following: 

https://medium.com/towards-data-science/assigning-panel-data-to-training-testing-and-validation-groups-for-machine-learning-models-7017350ab86e

Create a unique list of serial numbers

In [30]:
df_sample=df.select([c for c in df.columns if c in ['SERIAL_NUMBER']])
df_sample=df_sample.dropDuplicates(['SERIAL_NUMBER'])
df_sample=df_sample.withColumn('wookie', rand())


Randomly assign the serial numbers to either the Training, Testing or Validation group.

In [31]:
df_sample = df_sample.withColumn("GROUP", when(df_sample.wookie <= 0.33333333,"TRAINING")
                                 .when(df_sample.wookie >=0.6666666667,"TESTING")
                                 .otherwise("VALIDATION"))

Append the group to the original data frame

In [32]:
df_sample = df_sample.withColumnRenamed("SERIAL_NUMBER","DOODEE")
df_sample = df_sample.drop("wookie")


df=df.join(df_sample,(((df_sample.DOODEE) ==  (df.SERIAL_NUMBER))),"inner")
df = df.drop("DOODEE")

Check the data.  

We have 11,000 observations in the failure window.

In [33]:
z=df.groupBy("TARGET").count()
z.show()

+------+------+
|TARGET| count|
+------+------+
|     1| 11000|
|     0|534127|
+------+------+



1000 Failures

In [34]:
z=df.groupBy("FAILURE").count()
z.show()

+-------+------+
|FAILURE| count|
+-------+------+
|    0.0|544127|
|    1.0|  1000|
+-------+------+



An appropriate allocation among the training, testing and validation groups.

In [35]:
z=df.groupBy("GROUP").count()
z.show()

+----------+------+
|     GROUP| count|
+----------+------+
|   TESTING|177967|
|VALIDATION|178654|
|  TRAINING|188506|
+----------+------+



### 5.0 Prepare the data for the model <a id="50"></a>

Reformat features

In [36]:
from pyspark.sql.types import *
df = df.withColumn("MODEL_FAIL_CNT", df["MODEL_FAIL_CNT"].cast(DecimalType(12,6)))
df = df.withColumn("MODEL_FAIL_RATE", df["MODEL_FAIL_RATE"].cast(DecimalType(12,6)))


In [37]:
df = df.withColumn("MODEL_FAIL_TOTAL", df["MODEL_FAIL_TOTAL"].cast(DecimalType(12,6)))
df = df.withColumn("MANU_FAIL_RATE", df["MANU_FAIL_RATE"].cast(DecimalType(12,6)))
df = df.withColumn("MANU_FAIL_CNT", df["MANU_FAIL_CNT"].cast(DecimalType(12,6)))
df = df.withColumn("MANU_FAIL_TOTAL", df["MANU_FAIL_TOTAL"].cast(DecimalType(12,6)))
df = df.withColumn("REALLOCATED_SECTOR_COUNT_N_MOD", df["REALLOCATED_SECTOR_COUNT_N_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("REPORTED_UNCORRECTABLE_ERRORS_N_MOD", df["REPORTED_UNCORRECTABLE_ERRORS_N_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("COMMAND_TIMEOUT_N_MOD", df["COMMAND_TIMEOUT_N_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("CURRENT_PENDING_SECTOR_COUNT_N_MOD", df["CURRENT_PENDING_SECTOR_COUNT_N_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("POWER_ON_HOURS_N_MOD", df["POWER_ON_HOURS_N_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("REALLOCATED_SECTOR_COUNT_R_MOD", df["REALLOCATED_SECTOR_COUNT_R_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("REPORTED_UNCORRECTABLE_ERRORS_R_MOD", df["REPORTED_UNCORRECTABLE_ERRORS_R_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("COMMAND_TIMEOUT_R_MOD", df["COMMAND_TIMEOUT_R_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("CURRENT_PENDING_SECTOR_COUNT_R_MOD", df["CURRENT_PENDING_SECTOR_COUNT_R_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("POWER_ON_HOURS_R_MOD", df["POWER_ON_HOURS_R_MOD"].cast(DecimalType(12,6)))
df = df.withColumn("REALLOCATED_SECTOR_COUNT_N_MAN", df["REALLOCATED_SECTOR_COUNT_N_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("REPORTED_UNCORRECTABLE_ERRORS_N_MAN", df["REPORTED_UNCORRECTABLE_ERRORS_N_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("COMMAND_TIMEOUT_N_MAN", df["COMMAND_TIMEOUT_N_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("CURRENT_PENDING_SECTOR_COUNT_N_MAN", df["CURRENT_PENDING_SECTOR_COUNT_N_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("POWER_ON_HOURS_N_MAN", df["POWER_ON_HOURS_N_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("REALLOCATED_SECTOR_COUNT_R_MAN", df["REALLOCATED_SECTOR_COUNT_R_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("REPORTED_UNCORRECTABLE_ERRORS_R_MAN", df["REPORTED_UNCORRECTABLE_ERRORS_R_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("COMMAND_TIMEOUT_R_MAN", df["COMMAND_TIMEOUT_R_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("CURRENT_PENDING_SECTOR_COUNT_R_MAN", df["CURRENT_PENDING_SECTOR_COUNT_R_MAN"].cast(DecimalType(12,6)))
df = df.withColumn("POWER_ON_HOURS_R_MAN", df["POWER_ON_HOURS_R_MAN"].cast(DecimalType(12,6)))

Fill missings with -99.  Note this will work because we are using a non-parametric model.

In [38]:
df=df.na.fill(value=-99)

Define three data sets based on allocation to Training, Testing or Validation group.

In [39]:
df_training=df.filter(df.GROUP =='TRAINING')
df_testing=df.filter(df.GROUP =='TESTING')
df_validation=df.filter(df.GROUP =='VALIDATION')

Drop features that are non-predictive, text, or deriviatives of the predicted variable.

In [40]:
builder=df_training
builder = builder.drop('DATE','SERIAL_NUMBER','MODEL','FAILURE','ROW','MANUFACTURER','GROUP','FAILURE_DATE','MODELM')

Define the feature names.

In [41]:
feature_names = builder.drop('TARGET')

Prep the data for modeling and scoring on the validation and testing data sets.

In [42]:
features = feature_names.columns
va_train = VectorAssembler(inputCols = features, outputCol='features')

In [43]:
va_test=VectorAssembler(inputCols = features, outputCol='features')

va_valid=VectorAssembler(inputCols = features, outputCol='features')

In [44]:
df_training = va_train.transform(df_training)
df_validation=va_valid.transform(df_validation)
df_testing=va_test.transform(df_testing)
builder = va_train.transform(builder)



In [45]:
builder = builder.select(['features', 'TARGET'])


### 6.0 Build the model <a id="60"></a>

In [46]:
gbtc = GBTClassifier(labelCol="TARGET", maxIter=20)
gbtc = gbtc.fit(builder)


### 7.0 Evaluate model <a id="70"></a>

Evaluate the model with non-time-dependent stats like accuracy and F1

In [47]:
pred_tr = gbtc.transform(df_training)
pred_v = gbtc.transform(df_validation)
pred_te = gbtc.transform(df_testing)


In [48]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="TARGET",predictionCol="prediction", metricName=("accuracy"))
accuracy = evaluator.evaluate(pred_tr)
print("Training Accuracy = %g" % (accuracy))

Training Accuracy = 0.97877


In [49]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="TARGET",predictionCol="prediction", metricName=("accuracy"))
accuracy = evaluator.evaluate(pred_te)
print("Testing Accuracy = %g" % (accuracy)) 

Testing Accuracy = 0.980488


In [50]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="TARGET",predictionCol="prediction", metricName=("accuracy"))
accuracy = evaluator.evaluate(pred_v)
print("Validation Accuracy = %g" % (accuracy))

Validation Accuracy = 0.978154


In [51]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="TARGET",predictionCol="prediction", metricName=("f1"))
f1v = evaluator.evaluate(pred_v)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="TARGET",predictionCol="prediction", metricName=("f1"))
f1te = evaluator.evaluate(pred_te)



# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="TARGET",predictionCol="prediction", metricName=("f1"))
f1tr = evaluator.evaluate(pred_tr)


print( "Training F1 = %g" % (f1tr))
print( "Testing F1 = %g" % (f1te))
print( "Validation F1 = %g" % (f1v))

Training F1 = 0.974134
Testing F1 = 0.97303
Validation F1 = 0.971582


The results look very promising, but we definitely have more evaluation to do.  

### 8.0 Format the scored data for further evaluation <a id="80"></a>

Append the scored data into one dataframe

In [66]:
pred = pred_tr.union(pred_v)
pred =pred.union(pred_te)

Select relevant fields

In [67]:
pred.show(1)

+----------+-------------+-----------+-----------------+-------+--------------------------+-------------------------------+-----------------+------------------------------+----------------+--------------------------+-------------------------------+-----------------+------------------------------+----------------+------------+---+----------------------------+---------------------------------+-------------------+--------------------------------+------------------+----------------------------+---------------------------------+-------------------+--------------------------------+------------------+----------------------------+---------------------------------+-------------------+--------------------------------+------------------+----------------------------+---------------------------------+-------------------+--------------------------------+------------------+----------------------------+---------------------------------+-------------------+--------------------------------+-------------

In [68]:
pred=pred.select("DATE","FAILURE","SERIAL_NUMBER","TARGET","GROUP","probability","FAILURE_DATE")

Convert probablity so that it is usable

In [69]:
pred = pred.withColumn("probability",col("probability").cast(("string")))
predy = pred.withColumn("left", substring_index(pred.probability, ',', 1))
charReplace = udf(lambda x: x.replace(u'[',''))
predy=predy.withColumn('left',charReplace('left'))

predy = predy.withColumn("P_0",col("left").cast(("decimal(38,37)")))
predy = predy.withColumn("P_1",1-predy.P_0)

### 9.0 Export the data for use in Step 6 <a id="90"></a>

Select Relevant fields

In [71]:
predy=predy.select("DATE","SERIAL_NUMBER","FAILURE","TARGET","GROUP","P_0","P_1","FAILURE_DATE")

Convert to Pandas

In [72]:
df_pred = predy.toPandas()

Convert to csv

In [73]:
df_pred.head()

,DATE,SERIAL_NUMBER,FAILURE,TARGET,GROUP,P_0,P_1,FAILURE_DATE
0,2020-07-03,S301KWK5,0.0,0,TRAINING,0.9522802077717716000000000000000000000,0.047719792228228400000000000000000000,2020-11-03
1,2020-06-17,ZA13YPR7,0.0,0,TRAINING,0.9314597082268131000000000000000000000,0.068540291773186900000000000000000000,2020-12-02
2,2020-04-21,57RFWNHHT,0.0,0,TRAINING,0.9354118328476536000000000000000000000,0.064588167152346400000000000000000000,2020-07-27
3,2020-02-24,S301KWK5,0.0,0,TRAINING,0.9522802077717716000000000000000000000,0.047719792228228400000000000000000000,2020-11-03
4,2020-03-12,S301KWK5,0.0,0,TRAINING,0.9522802077717716000000000000000000000,0.047719792228228400000000000000000000,2020-11-03


In [74]:
df_pred=df_pred.to_csv('pred.csv',index=False)

Define Object storage credentials.

In [75]:
# The code was removed by Watson Studio for sharing.

In [76]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

Export csv to object storage

In [77]:
cos.upload_file(Filename='pred.csv',Bucket=credentials['BUCKET'],Key='pred.csv')

All data used in this notebook is the property of BackBlaze.com.

For questions regarding use of data please see the following website. https://www.backblaze.com/b2/hard-drive-test-data.html